Following the textbook, maximize the precision of these datasets.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
FILENAME = 'NBADATA.csv'

In [3]:
#retain relevant columns. 
data = pd.read_csv(FILENAME) 
data['3P%'] = np.divide(data['3P'].values,data['3PA'].values) 
del data['3P'],data['3PA']
data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
del data['FG'],data['FGA']
data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
del data['Unnamed: 0'],data['PLUS_MINUS'],data['TOTAL']
del data['FT'],data['FTA']
del data['OU']
#del data['Team']
#data = pd.get_dummies(data)

In [4]:
teams = data.Team.unique()

In [5]:
#iterate over those teams, make a rolling window over n games. 
N_GAMES = 5
nba_data = pd.DataFrame([])
for team in teams:
    team_data = data.loc[data['Team'] == team]  #this contains the box score of every team game from 2013 to 2018. 
    stuff_to_turn_into_avgs = ['OR', 'DR', 'TOT', 'PF', 'ST', 'TO', 'BL', '3P%', 'FG%', 'FT%','PTS']
    #if seasons are the same, do this here, use the GAME_ID signifier. 
    for col in team_data.columns:
        if col in stuff_to_turn_into_avgs:
            #split each season up here, 
            team_data['Rolling ' + col] = team_data[col].rolling(window=N_GAMES).mean().shift(1)
            del team_data[col]
                
    nba_data =  nba_data.append(team_data)
           # df = pd.concat([road_df,home_df],axis=1)
#reorganize the dataset. 
nba_data_splits = nba_data.sort_values(by = ['GAME_ID', 'Home'], ascending=[True, True])
nba_data_splits.dropna(inplace=True)  #null values come with rolling means, drop those now. 

#delete columns no longer of use, ie team name etc. Can consider keeping team name and see if helps chances. 
del nba_data_splits['GAME_ID'],nba_data_splits['Home'],nba_data_splits['Away'],nba_data_splits['Date']
del nba_data_splits['Team']

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
#now align the box scores so its one big one for each game, home team and road teams. 

road_df = nba_data_splits.iloc[::2]
home_df = nba_data_splits.iloc[1::2]
for col in nba_data_splits.columns:
    road_df['road_' + col] = road_df[col]
    home_df['home_' + col] = home_df[col]
    
    del road_df[col],home_df[col]

home_df.reset_index(inplace=True)
road_df.reset_index(inplace=True)

#merged into a dataframe here. 
df = pd.concat([road_df,home_df],axis=1)
del df['index']

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
y = df['road_SPREAD']

del df['road_SPREAD'],df['home_SPREAD']

In [8]:
df

,road_Rolling OR,road_Rolling DR,road_Rolling TOT,road_Rolling PF,road_Rolling ST,road_Rolling TO,road_Rolling BL,road_Rolling PTS,road_Rolling 3P%,road_Rolling FG%,road_Rolling FT%,home_Rolling OR,home_Rolling DR,home_Rolling TOT,home_Rolling PF,home_Rolling ST,home_Rolling TO,home_Rolling BL,home_Rolling PTS,home_Rolling 3P%,home_Rolling FG%,home_Rolling FT%
0,12.0,30.4,26.6,25.6,9.2,14.4,5.0,113.2,0.351003,0.477664,0.724035,6.8,27.4,28.6,24.0,8.6,15.8,3.8,104.8,0.458438,0.501633,0.771443
1,10.0,35.0,22.4,20.0,6.2,14.8,5.6,100.8,0.394119,0.416175,0.729286,11.8,35.8,17.6,21.6,6.8,18.6,4.6,109.4,0.340226,0.498713,0.712209
2,11.4,29.2,17.8,21.4,7.4,13.0,6.2,90.2,0.274762,0.421881,0.645434,14.2,31.4,16.4,24.0,5.2,14.6,3.2,94.0,0.325206,0.433487,0.702972
3,9.4,36.4,21.4,18.4,6.0,17.6,10.4,95.4,0.404230,0.452436,0.747334,9.4,29.8,15.8,23.4,8.8,18.6,5.4,89.4,0.273654,0.454401,0.784297
4,10.4,38.0,20.0,22.0,8.4,16.6,5.4,103.4,0.413664,0.474791,0.696173,11.2,31.8,17.6,23.8,10.2,15.2,3.8,90.6,0.299325,0.410580,0.781355
5,10.4,35.0,23.8,17.8,9.2,19.2,3.8,104.4,0.335031,0.471733,0.662786,15.4,29.6,17.6,23.8,6.4,19.4,3.8,90.0,0.214177,0.416141,0.694815
6,11.6,31.6,24.0,21.8,10.0,15.6,3.6,110.0,0.406575,0.459247,0.835104,11.0,32.8,22.8,18.0,10.4,15.4,2.8,102.8,0.303788,0.406218,0.788802
7,9.0,35.0,23.0,22.6,6.2,15.8,5.6,97.4,0.388996,0.404113,0.747381,12.2,29.6,21.0,22.6,9.2,14.0,8.8,96.4,0.471251,0.432727,0.815500
8,8.8,36.8,27.8,26.4,10.6,20.0,7.2,110.8,0.465826,0.504002,0.715916,8.4,32.2,24.2,20.0,7.0,13.8,4.8,99.6,0.382619,0.475608,0.719524
9,10.2,33.8,18.6,20.2,9.2,17.4,5.4,97.4,0.342011,0.457137,0.715909,16.8,27.4,16.2,22.4,7.8,18.4,3.6,85.0,0.229632,0.383825,0.689481


In [9]:
#scale and split the data here. 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#this is done in a relatively un-pythonic way. Can also all be in one line! 
scaler = MinMaxScaler()

scaler.fit(df.values)
X = scaler.transform(df.values)
X_train,X_test, y_train, y_test = train_test_split(X,y.values,random_state = 69)

# Trying some simple stuff right here. Want to use polynomial ordering, besides that go back to the copy 1 version. 

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
logreg = LinearRegression()
logreg.fit(X_train,y_train)
print(logreg.score(X_train,y_train))

print(logreg.score(X_test,y_test))  #bad!

0.322806492728
0.308692424596


In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import precision_score

poly_transformer = PolynomialFeatures(degree=2,interaction_only=True,include_bias=False)
X_train_poly = poly_transformer.fit_transform(X_train)

X_test_poly = poly_transformer.fit_transform(X_test)


from sklearn.decomposition import PCA
principal_features = 3
pca = PCA(n_components = principal_features)

pca.fit(X_train)
X_train_pca = pca.transform(X_train)  #turns it into a two feature dataset. 
X_test_pca = pca.transform(X_test)

In [13]:
from sklearn.linear_model import LinearRegression
logreg = LinearRegression()
print("Normal")
logreg.fit(X_train,y_train)
print(logreg.score(X_train,y_train))

print(logreg.score(X_test,y_test))
print("")
print("Poly")
logreg.fit(X_train_poly,y_train)
print(logreg.score(X_train_poly,y_train))

print(logreg.score(X_test_poly,y_test))


print("")
print("PCA")
logreg.fit(X_train_pca,y_train)
print(logreg.score(X_train_pca,y_train))

print(logreg.score(X_test_pca,y_test))

Normal
0.322806492728
0.308692424596

Poly
0.3779012633
0.3025079605

PCA
0.224828582675
0.230910206417


In [14]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor()

print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)
#print('precision= ',precision_score(y_test,y_pred))

print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)
#print('precision= ',precision_score(y_test,y_pred))

print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)
#print('precision= ',precision_score(y_test,y_pred))

normal
0.322282193473
0.306498625125

poly
0.320592419948
0.304866268216

pca
0.224106055592
0.230909043135


In [15]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)

print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)


print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)


normal
0.863046665985
0.233467249422

poly
0.865079215683
0.256472546433

pca
0.840267513612
0.108758058161


In [16]:
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()

print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)


print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)


print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)


normal
0.452041002372
0.308587674512

poly
0.504412372959
0.308849910678

pca
0.301589015397
0.225008691373


In [17]:
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor()

print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)


print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)


print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)



normal
0.475094457957
0.172269462599

poly
0.465203001959
0.168016742027

pca
0.387522562611
0.100116287412


In [18]:
from sklearn.svm import SVR

model = SVR()


print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)


print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)


print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)



normal
0.280943476216
0.279491573436

poly
0.223561676891
0.226852024385

pca
0.219753306871
0.230892535401


In [19]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()


print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)


print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)


print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)



normal


ValueError: Unknown label type: (array([-17.5, -16.5, -16. , -15.5, -15. , -14.5, -14. , -13.5, -13. ,
       -12.5, -12. , -11.5, -11. , -10.5, -10. ,  -9.5,  -9. ,  -8.5,
        -8. ,  -7.5,  -7. ,  -6.5,  -6. ,  -5.5,  -5. ,  -4.5,  -4. ,
        -3.5,  -3. ,  -2.5,  -2. ,  -1.5,  -1. ,   0. ,   1. ,   1.5,
         2. ,   2.5,   3. ,   3.5,   4. ,   4.5,   5. ,   5.5,   6. ,
         6.5,   7. ,   7.5,   8. ,   8.5,   9. ,   9.5,  10. ,  10.5,
        11. ,  11.5,  12. ,  12.5,  13. ,  13.5,  14. ,  14.5,  15. ,
        15.5,  16. ,  16.5,  17. ,  17.5,  18. ,  18.5,  19. ,  19.5,
        20.5,  21. ,  22. ]),)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

print("normal")
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
y_pred = model.predict(X_test)
print('precision= ',precision_score(y_test,y_pred))

print()
print("poly")
model.fit(X_train_poly,y_train)
print(model.score(X_train_poly,y_train))
print(model.score(X_test_poly,y_test))
y_pred = model.predict(X_test_poly)
print('precision= ',precision_score(y_test,y_pred))

print()
print("pca")

model.fit(X_train_pca,y_train)
print(model.score(X_train_pca,y_train))
print(model.score(X_test_pca,y_test))
y_pred = model.predict(X_test_pca)
print('precision= ',precision_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import precision_score

print('precision= ',precision_score(y_test,y_pred))

In [ ]:
model = MLPClassifier()

model.fit(X_trainval,y_trainval)
model.n_layers_ = 4
print(model.n_layers_)
model.


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

X_trainval,X_test,y_trainval,y_test = train_test_split(X,y,random_state = 0)

X_train,X_val,y_train,y_val = train_test_split(X_trainval,y_trainval,random_state = 1)
#print size of datasets here

best_score = 0

for gamma in [.001,.01,.1,1,10,100]:
    for C in [.001,.01,.1,1,10,100]:
        svm = SVC(gamma=gamma,C=C)
        svm.fit(X_train,y_train)
        score = svm.score(X_val,y_val)
        if score > best_score:
            print("obtained score:", score)

            best_score = score
            best_parameters = {'C' : C,'gamma' : gamma}

svm = SVC(gamma=best_parameters['gamma'],C=best_parameters['C'])
svm.fit(X_trainval,y_trainval)
test_score = svm.score(X_test,y_test)
print("best params: ",best_parameters['gamma'],best_parameters['C'])
print("test score w/ best params: ", test_score)


Created best SVM, now optimize precision. 

In [ ]:
#Look at ordinary confusion matrix. 
from sklearn.metrics import confusion_matrix
y_pred = svm.predict(X_test)
confusion = confusion_matrix(y_test,y_pred)

print(confusion)

In [ ]:
588 /(588+559)

In [ ]:
#Now try lowering the threshold. 
y_pred_lower_threshold = svm.decision_function(X_test) >thresholds[close_zero]
confusion = confusion_matrix(y_test,y_pred_lower_threshold)
print(confusion)

In [ ]:
thresholds[close_zero]

Now time for the precision recall curve. 

In [ ]:
precision,recall,thresholds = precision_recall_curve(y_test,svm.decision_function(X_test))
close_zero = np.argmin(np.abs(thresholds))
plt.plot(precision[close_zero],recall[close_zero],'o')
plt.plot(precision,recall,label = 'precision recall curve')
plt.xlabel("precision")
plt.ylabel('recall')
plt.legend(loc='best')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
precision.ind[max(precision)]

In [ ]:
data = pd.DataFrame([])

data['Precision'] = precision
data['recall'] = recall
data['threshold'] = thresholds

In [ ]:
np.argmin(thresholds)

In [ ]:
len(precision)

In [ ]:
thresholds[0]

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test,y_pred_higher_threshold)

In [ ]:
confusion

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precision,recall, thresholds = precision_recall_curve(y_test)

In [ ]:
y_pred_higher_threshold = svm.decision_function(X_test) >-.8

In [ ]:
sum(y_pred_lower_threshold)/len(y_pred_lower_threshold)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X_trainval,X_test,y_trainval,y_test = train_test_split(X,y,random_state = 0)

X_train,X_val,y_train,y_val = train_test_split(X_trainval,y_trainval,random_state = 1)
#print size of datasets here

best_score = 0

for alpha in [.0001,.001,.01]:
    model = MLPClassifier(alpha=alpha)
    model.fit(X_train,y_train)
    score = model.score(X_val,y_val)
    if score > best_score:
        print("obtained score:", score)

        best_score = score
        best_parameters = {'alpha' : alpha}

model = MLPClassifier(alpha=best_parameters['alpha'])
model.fit(X_trainval,y_trainval)
test_score = model.score(X_test,y_test)
print("best params: ",best_parameters['alpha'])
print("test score w/ best params: ", test_score)
